In [29]:
import pandas as pd

#---Change the file name here!---
filePath1 = r'purchase_data2.json'
#filePath2 = r'purchase_data2.json'

Purchases_df = pd.read_json(filePath1)
#Purchases2_df = pd.read_json(filePath2)

Purchases_df.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,20,Male,93,Apocalyptic Battlescythe,4.49,Iloni35
1,21,Male,12,Dawne,3.36,Aidaira26
2,17,Male,5,Putrid Fan,2.63,Irim47
3,17,Male,123,Twilight's Carver,2.55,Irith83
4,22,Male,154,Feral Katana,4.11,Philodil43


In [30]:
# **Player Count**
# Get no. of unique player names. Be careful of players who purchased multiple times!
player_cnt = Purchases_df['SN'].nunique()

#Create table of player count, starting with turning player_cnt into series to get rid of aesthetic errors 
player_cnt_S = pd.Series(player_cnt)
player_cnt_df = pd.DataFrame(player_cnt_S)
player_cnt_df.columns = ['Total Players']

player_cnt_df

,Total Players
0,74


In [31]:
# **Purchasing Analysis (Total)**
'''* Number of Unique Items * Average Purchase Price * Total Number of Purchases * Total Revenue'''

uniq_item = Purchases_df['Item Name'].nunique()
avgPurPrice = Purchases_df['Price'].mean()
totlNumPurch = Purchases_df['Price'].count()
totlRev = avgPurPrice * totlNumPurch

PurchAnalys_df = pd.DataFrame({'Number of Unique Items':uniq_item, 'Average Price':avgPurPrice,\
                               'Number of Purchases':totlNumPurch, 'Total Revenue':totlRev}, index = [0])
PurchAnalys_df.reindex(columns = ['Number of Unique Items', 'Average Price',\
                                  'Number of Purchases', 'Total Revenue'])


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,63,2.924359,78,228.1


In [32]:
# **Gender Demographics**
'''* Percentage and Count of Male Players * Percentage and Count of Female Players 
* Percentage and Count of Other / Non-Disclosed'''

#initializing some useful variables
IsMale = Purchases_df['Gender']=='Male'
IsFemale = Purchases_df['Gender']=='Female'
IsNoGend = (Purchases_df['Gender']!='Male') & (Purchases_df['Gender']!='Female')
indexGen = ['Male', 'Female', 'Other/Non-Disclosed']

#getting count of all player genders, careful of repeat purchases
MaleCt =  Purchases_df[IsMale]['SN'].nunique()
FemCt = Purchases_df[IsFemale ]['SN'].nunique()
NAGendCt = Purchases_df[IsNoGend]['SN'].nunique()
GendCt = pd.Series([MaleCt, FemCt, NAGendCt], index=indexGen)

#getting % of all player genders
MalePerc = MaleCt/player_cnt #Purch_cnt ("total purchase count") from previous section **player count**
FemPerc = FemCt/player_cnt
NAPerc = NAGendCt/player_cnt
GendPerc = pd.Series([MalePerc, FemPerc, NAPerc], index=indexGen)

Gen_df = pd.DataFrame({'Percentage of Players':GendPerc, 'Total Count':GendCt})

Gen_df

,Percentage of Players,Total Count
Male,0.810811,60
Female,0.175676,13
Other/Non-Disclosed,0.013514,1


In [33]:
# **Purchasing Analysis (Gender)** 
'''Broken by gender:* Purchase Count * Average Purchase Price * Total Purchase Value * Normalized Totals'''

# I count multiple purchases by the same person 
MaleCt2 = Purchases_df[IsMale].count()
MaleCt2 = MaleCt2['SN']
FemCt2 = Purchases_df[IsFemale].count()
FemCt2 = FemCt2['SN']
NAGendCt2 = Purchases_df[IsNoGend].count()
NAGendCt2 = NAGendCt2['SN']
GendCt_s = pd.Series([MaleCt2, FemCt2, NAGendCt2], index=indexGen)

avgPurchPr_M = Purchases_df[IsMale]['Price'].mean()
avgPurchPr_F = Purchases_df[IsFemale]['Price'].mean()
avgPurchPr_N = Purchases_df[IsNoGend]['Price'].mean()
GendPurchAvg_s = pd.Series([avgPurchPr_M, avgPurchPr_F, avgPurchPr_N], index = indexGen)

totlPurchV_M =  Purchases_df[IsMale]['Price'].sum()
totlPurchV_F = Purchases_df[IsFemale]['Price'].sum()
totlPurchV_N = Purchases_df[IsNoGend]['Price'].sum()
GendPurchTot_s = pd.Series([totlPurchV_M, totlPurchV_F, totlPurchV_N], index = indexGen)

#not sure what is meant by Normalized Totals. I'm guessing it's the average of the totals per person of each gender
#The count (...Ct) variables taken from **Gender Demographics** section. They only include unique SNs 
normTotl_M = totlPurchV_M / MaleCt
normTotl_F = totlPurchV_F / FemCt
normTotl_N = totlPurchV_N / NAGendCt
GendNormTot_s = pd.Series([normTotl_M, normTotl_F, normTotl_N], index = indexGen)

# Create new dataframe with all the data series put together
GendPurch_df = pd.DataFrame({'Purchase Count':GendCt_s, 'Average Purchase Price':GendPurchAvg_s,\
                            'Total Purchase Value':GendPurchTot_s, 'Normalized Totals':GendNormTot_s})
GendPurch_df.reindex(columns=['Purchase Count', 'Average Purchase Price', 'Total Purchase Value', 'Normalized Totals'])

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Male,64,2.884375,184.60,3.076667
Female,13,3.183077,41.38,3.183077
Other/Non-Disclosed,1,2.120000,2.12,2.120000


In [34]:
#**Age Demographics**
''' The below each broken into bins of 4 years (i.e. &lt;10, 10-14, 15-19, etc.) 
* Purchase Count * Average Purchase Price * Total Purchase Value * Normalized Totals'''

#Purchases_df['Age']: max==40, min==7
#create bins and append 'Age Range' to Purchases_df
bins=[0, 10, 15, 20, 25, 30, 35, 40, 150]
labels=['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']
Purchases_df['Age Range'] = pd.cut(Purchases_df['Age'], bins=bins, right=False, labels=labels)
PurchAges = Purchases_df.groupby('Age Range')

#Purchase Count
PurchAgeCt = PurchAges['SN'].count()
#Average Purchase Price
avgPurchP = PurchAges['Price'].mean()
#Total Purchase Value
totlPurchV = PurchAges['Price'].sum()
#Normalized Totals. I'm guessing this to be (total/# of players) for each age group
normPurchV = totlPurchV / PurchAges['SN'].nunique()

#concatenate ALL the columns!!!
AgePurch_df = pd.concat({'Purchase Count':PurchAgeCt, 'Average Purchase Price':avgPurchP,\
                            'Total Purchase Value':totlPurchV, 'Normalized Totals':normPurchV}, axis=1)
#fix column order
AgePurch_df.reindex(columns = ['Purchase Count', 'Average Purchase Price', 'Total Purchase Value', 'Normalized Totals'])

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Age Range,,,,
<10,5,2.764000,13.82,2.764000
10-14,3,2.986667,8.96,2.986667
15-19,11,2.764545,30.41,2.764545
20-24,36,3.024722,108.89,3.202647
25-29,9,2.901111,26.11,3.263750
30-34,7,1.984286,13.89,2.315000
35-39,6,3.561667,21.37,3.561667
40+,1,4.650000,4.65,4.650000


In [35]:
# **Top Spenders**
''' Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
* SN, Purchase Count, Average Purchase Price, Total Purchase Value '''
#group by each player
TopSpends = Purchases_df.groupby('SN')

#getting Purchase Count, Average Purchase Price, Total Purchase Value 
PurchCt_TS = TopSpends['Price'].count()
avgPr_TS = TopSpends['Price'].mean()
totlPurchV_TS = TopSpends['Price'].sum()

#concat into a df, sort by largest total purchases by each player, and fix column order
TopS_df = pd.concat({'Purchase Count': PurchCt_TS, 'Average Purchase Price': avgPr_TS,\
                     'Total Purchase Value':totlPurchV_TS}, axis=1)
TopS_df = TopS_df.sort_values(by='Total Purchase Value', ascending=False)
TopS_df = TopS_df.reindex(columns=['Purchase Count', 'Average Purchase Price', 'Total Purchase Value'])
TopS_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Sundaky74,2,3.705,7.41
Aidaira26,2,2.565,5.13
Eusty71,1,4.810,4.81
Chanirra64,1,4.780,4.78
Alarap40,1,4.710,4.71


In [36]:
# **Most Popular Items**
'''Identify the 5 most popular items by purchase count, then list (in a table):
* Item ID, Item Name, Purchase Count, Item Price, Total Purchase Value'''
#initial group by
Item_GBy = Purchases_df.groupby('Item ID')

#Getting Item Name, Purchase Count, Item Price, Total Purchase Value
#ItmName and ItmPr only works when a function is called to them 
ItmName = Item_GBy['Item Name'].unique()
PurchCt = Item_GBy['Price'].count()
ItmPr = Item_GBy['Price'].unique()
totlItemV = Item_GBy['Price'].sum()

#Form vars into new dataframe, sort first by Purchase count, then by total purchase value, then format & view
PopItm_df = pd.DataFrame({'Item Name':ItmName, 'Purchase Count': PurchCt,\
                      'Item Price':ItmPr, 'Total Purchase Value': totlItemV})
PopItm_df = PopItm_df.sort_values(['Purchase Count', 'Total Purchase Value'], ascending=False)
PopItm_df = PopItm_df.reindex(columns=['Item Name', 'Purchase Count', 'Item Price', 'Total Purchase Value'])
PopItm_df.head()

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
94,[Mourning Blade],3,[3.64],10.92
117,"[Heartstriker, Legacy of the Light]",2,[4.71],9.42
93,[Apocalyptic Battlescythe],2,[4.49],8.98
90,[Betrayer],2,[4.12],8.24
154,[Feral Katana],2,[4.11],8.22


In [37]:
# **Most Profitable Items**
''' Identify the 5 most profitable items by total purchase value, then list (in a table):
* Item ID, Item Name, Purchase Count, Item Price, Total Purchase Value '''

#Using variables & DF from previous section **Most Popular Items**
ProfItm_df = PopItm_df.sort_values('Total Purchase Value', ascending=False)
ProfItm_df.head()


,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
94,[Mourning Blade],3,[3.64],10.92
117,"[Heartstriker, Legacy of the Light]",2,[4.71],9.42
93,[Apocalyptic Battlescythe],2,[4.49],8.98
90,[Betrayer],2,[4.12],8.24
154,[Feral Katana],2,[4.11],8.22
